# Inspect FermiHarmonics Observables

Load and visualize observables (A0, A1, B1) from the new Python-friendly HDF5 format.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from pathlib import Path

In [ ]:
# Load data from new format (uniform grid)
data_file = Path("results/example_single_run/data/observables_sample.h5")

with h5py.File(data_file, "r") as f:
    # Load 2D arrays (ny, nx) on uniform grid
    A0 = f["a0"][:]
    A1 = f["a1"][:]
    B1 = f["b1"][:]
    x = f["x"][:]  # 1D coordinate array
    y = f["y"][:]  # 1D coordinate array
    
    # Metadata
    time = f.attrs["time"]
    nx = f.attrs["nx"]
    ny = f.attrs["ny"]

print(f"Time: {time}")
print(f"Grid: {nx} x {ny}")
print(f"A0 shape: {A0.shape}")


# Create meshgrid for plotting
X, Y = np.meshgrid(x, y)

In [ ]:
# pcolormesh plot of A0 (density)
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)
pcm = ax.pcolormesh(X, Y, A0, cmap='viridis', shading='auto')
plt.colorbar(pcm, ax=ax)
ax.set_xlabel('$x/L$')
ax.set_ylabel('$y/L$')
ax.set_title(r'Component $A_0$ (density)')
ax.set_aspect('equal')
plt.tight_layout()

In [ ]:
# pcolormesh plot of A1 (x-current)
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)

# Use nanmax to handle NaN values properly
vmax = np.nanmax(np.abs(A1))
print(f"A1 range: [{np.nanmin(A1):.4f}, {np.nanmax(A1):.4f}], max|A1|: {vmax:.4f}")
print(f"NaN count: {np.sum(np.isnan(A1))}/{A1.size}")

pcm = ax.pcolormesh(X, Y, A1, cmap='bwr', shading='auto', vmin=-vmax, vmax=vmax)
plt.colorbar(pcm, ax=ax)
ax.set_xlabel('$x/L$')
ax.set_ylabel('$y/L$')
ax.set_title(r'Component $A_1 \sim j_x$')
ax.set_aspect('equal')
plt.tight_layout()

In [ ]:
# Current magnitude with streamplot
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)
current_mag = np.sqrt(A1**2 + B1**2)
pcm = ax.pcolormesh(X, Y, current_mag, cmap='coolwarm', shading='auto')

# Add streamlines
stream = ax.streamplot(X[0, :], Y[:, 0], A1, B1, color='k', linewidth=0.5, density=0.9, arrowsize=0.8)

plt.colorbar(pcm, ax=ax, label=r'$\sqrt{A_1^2 + B_1^2}$')
ax.set_xlabel('$x/L$')
ax.set_ylabel('$y/L$')
ax.set_title('Current Density Vector Field')
ax.set_aspect('equal')
plt.tight_layout()

In [ ]:
# Line plot: B1 at a horizontal cut
y_cut = 0.4

# Find nearest y index
y_idx = np.argmin(np.abs(y - y_cut))
y_actual = y[y_idx]

fig, ax = plt.subplots(figsize=(6, 4), dpi=100)
ax.plot(x, B1[y_idx, :], 'o-', markersize=3, linewidth=1)
ax.set_xlabel('$x/L$')
ax.set_ylabel(r'$B_1$')
ax.set_title(f'$B_1$ at $y/L = {y_actual:.3f}$ (requested {y_cut:.2f})')
ax.grid(True, alpha=0.3)
plt.tight_layout()

In [ ]:
# Summary: all three observables side-by-side
fig, axes = plt.subplots(1, 3, figsize=(18, 6), dpi=100)

for idx, (data, name, cmap) in enumerate([(A0, 'A0 (density)', 'viridis'),
                                            (A1, 'A1 (j_x)', 'bwr'),
                                            (B1, 'B1 (j_y)', 'bwr')]):
    ax = axes[idx]
    vmax = np.abs(data).max() if 'bwr' in cmap else None
    vmin = -vmax if vmax else None
    
    pcm = ax.pcolormesh(X, Y, data, cmap=cmap, shading='auto', vmin=vmin, vmax=vmax)
    ax.set_title(name)
    ax.set_xlabel('$x/L$')
    ax.set_ylabel('$y/L$')
    ax.set_aspect('equal')
    plt.colorbar(pcm, ax=ax, shrink=0.8)

plt.suptitle(f'Observables at t={time:.3f}', y=1.02)
plt.tight_layout()